In [9]:
import pandas as pd, numpy as np, json, os
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
m=json.loads(open('ignore/member_timelines.json','r').read())

In [8]:
m[10]

{'start': '1992-10-28 00:00:00',
 'end': '1996-11-22 00:00:00',
 'Name': 'ANTAL István',
 'duration': '1486',
 'Simple': 'Antal Istvan | 1948-06-23',
 'Csoportok': '⏫ Gazdaság, ipar',
 'Csoportok0': '⏫ Gazdaság, ipar    ',
 'Csoportok1': '⏫ Gazdaság, ipar',
 'Típus': 'Bizottság'}

In [274]:
# m=m[:1000]

In [ ]:
data={'Ország':{},'Bizottság':{},'Párt':{},'Megye':{}}
attention={}
for k,i in enumerate(m):
    if k%1000==0: print(k/len(m)*100,'%')
    name=i['Simple']
    if i['Típus'] in list(data.keys()):
        if i['Csoportok'] not in data[i['Típus']]:data[i['Típus']][i['Csoportok']]={}
        for t in pd.date_range(i['start'],i['end']):
            st=str(t)[:10]
            if st not in data[i['Típus']][i['Csoportok']]:data[i['Típus']][i['Csoportok']][st]=set()
            data[i['Típus']][i['Csoportok']][st].add(name)            
            if name not in attention:attention[name]={}
            if st not in attention[name]:attention[name][st]=0
            attention[name][st]+=1

0.0 %
3.308409978164494 %
6.616819956328988 %


In [ ]:
def anneal(t,p=7):
    #pd.to_datetime('2020-01-01')-pd.to_datetime('1989-01-01')=11322
    return (2-(pd.to_datetime('2020-01-01')-pd.to_datetime(t)).days/11322)**p
    
tanneal={}
anneal2020=anneal(pd.to_datetime('2020'))
for t in pd.date_range('1990','2020'):
    tanneal[t]=anneal(t)/anneal2020+0.1
    tanneal[str(t)[:10]]=tanneal[t]
plt.subplots(1,1,figsize=(5,3))
plt.grid()
plt.plot(tanneal.keys(),tanneal.values())
plt.ylabel('Elavulási együttható')
plt.show()

In [ ]:
edges={}
points={'Ország':3,'Bizottság':4,'Párt':2,'Megye':1}
for tipus in data:
    for csoport in data[tipus]:
        print(csoport)
        for t in data[tipus][csoport]:
            names=list(data[tipus][csoport][t])
            for r in range(len(names)):
                name=names[r]
                for p in range(r+1,len(names)):
                    name2=names[p]
                    nset=tuple(np.sort([name,name2]))
                    v=points[tipus]
                    if nset not in edges:edges[nset]={'anneal':0,'values':0}
                    edges[nset]['values']+=v
                    edges[nset]['anneal']+=v*tanneal[t]

Save

In [ ]:
import pickle

In [ ]:
def save_obj(obj, name ):
    with open('ignore/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('ignore/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
save_obj(edges,'plotter_name_edges')

In [82]:
edges=load_obj('plotter_name_edges')

Prune

In [ ]:
df=pd.DataFrame(edges).T

In [ ]:
all_nodes=set(df.reset_index()['level_0']).union(df.reset_index()['level_1'])

In [ ]:
min_baratsag=500
df2=df[df['anneal']>min_baratsag]

In [ ]:
df2['regi']=df2['anneal']/df2['values']

In [ ]:
df2=df2['anneal'].reset_index()

In [ ]:
df2.columns=['source','target','weight']

In [ ]:
nodes={}
for i in all_nodes:
    if i not in nodes:
        nodes[i]={'pr':0,'bc':0,'cf':0}

In [ ]:
G=nx.Graph(df2)

In [ ]:
r = nx.pagerank(G,weight='weight')
mr=np.median(list(r.values()))
for i in r:
    nodes[i]['pr']=r[i]/mr

In [ ]:
Gc = max(nx.connected_component_subgraphs(G), key=len)
r = nx.current_flow_closeness_centrality(Gc,weight='weight')
mr=np.median(list(r.values()))
for i in r:
    nodes[i]['cf']=r[i]/mr

In [ ]:
links={}
for i in G.edges(data=True):
    name=i[0]
    name2=i[1]
    z=i[2]['weight']
    if name not in links:links[name]={}
    if name2 not in links:links[name2]={}
    links[name][name2]=z
    links[name2][name]=z

In [ ]:
#save 
network={'nodes':nodes,'links':links}
print(len(nodes),'nodes and',len(G.edges()),'edges')

In [ ]:
open('ignore/plotter_name.json','w').write(json.dumps(network))

In [ ]:
import zipfile

with zipfile.ZipFile("json/plotter_name.zip", "w", compression=zipfile.ZIP_DEFLATED) as zf:
    zf.write("ignore/plotter_name.json",'plotter_name.json')